In [ ]:
import glob as glob

import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def read_vars(fName, vName):
    ds = xr.open_dataset(fName)
    var = ds[vName]
    return var

In [ ]:
data_rootPath = "/scratch3/NCEPDEV/climate/Jiande.Wang/working/scratch/C384-v17/OCEAN/"
expNames = ["B1", "B2", "B3"]
vNames = ['Temp', 'Salt']
depth_plot =31

# path where figures will be saved
out_dir = "/home/Santha.Akella/for_folks/JiandeW/Sep2025/figs/"

In [ ]:
# load data, calculate mean (dirty! not grid weighted.), save it to a file.

for iexp, expName in enumerate(expNames):
    # Load data
    dPath = data_rootPath+f"{expNames[iexp]}/gdas.*/*/model/ocean/history/"
    fNames = sorted(glob.glob(dPath + f"gdas.ocean.t*z.inst.f006.nc"))
    ds = xr.open_mfdataset(fNames)
    print(f'Gathered data for {expName}')
    # calculate mean
    for iVar, vName in enumerate(vNames):
        meanVal=ds[vName].isel(z_l=depth_plot).mean(('xh','yh')).compute()
        meanVal['time']=meanVal.indexes['time'].to_datetimeindex() # needed because Julian cal doesn't play nice with plotting.
        # save file
        fName = out_dir + 'glb_mean_' + f'{vName}_{expName}.nc'
        meanVal.to_netcdf(fName)
        print(f'Saved {fName}')

## Load the calculated data and plot

In [ ]:
for iVar, vName in enumerate(vNames):
    fig = plt.figure(figsize=[10, 6], clear=True, dpi=120)
    ax=plt.subplot(111)
    for iexp, expName in enumerate(expNames):
        fName = out_dir + 'glb_mean_' + f'{vName}_{expName}.nc'
        ds = xr.open_dataset(fName)
        ds[vName].plot(ax=ax, label=f'{expName}')
    plt.legend(loc=1)
    ax.set_title(f'depth=100 m')
    figName = out_dir + 'glb_mean_tSeries' + f'{vName}.png'
    plt.savefig(figName, bbox_inches='tight')
    plt.close('all')